## Classification and Differencing of Two Canopy Height Models for the Teakettle Experimental Watershed

The goal of this program is to read in two canopy height models, classify them by height, and perform a change detection.

Nicole Abib nicole.abib@duke.edu

In [ ]:
# Check python version
import sys
sys.version

In [ ]:
# Import required libraries
import numpy as np
import gdal, osr, copy
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# %load ../functions/neon_aop_lidar_python_functions/raster2array.py
def raster2array(geotif_file):
    metadata = {}
    dataset = gdal.Open(geotif_file)
    metadata['array_rows'] = dataset.RasterYSize
    metadata['array_cols'] = dataset.RasterXSize
    metadata['bands'] = dataset.RasterCount
    metadata['driver'] = dataset.GetDriver().LongName
    metadata['projection'] = dataset.GetProjection()
    metadata['geotransform'] = dataset.GetGeoTransform()
    
    mapinfo = dataset.GetGeoTransform()
    metadata['pixelWidth'] = mapinfo[1]
    metadata['pixelHeight'] = mapinfo[5]

    xMin = mapinfo[0]
    xMax = mapinfo[0] + dataset.RasterXSize/mapinfo[1]
    yMin = mapinfo[3] + dataset.RasterYSize/mapinfo[5]
    yMax = mapinfo[3]
    
    metadata['extent'] = (xMin,xMax,yMin,yMax)
    
    raster = dataset.GetRasterBand(1)
    array_shape = raster.ReadAsArray(0,0,metadata['array_cols'],metadata['array_rows']).astype(np.float).shape
    metadata['noDataValue'] = raster.GetNoDataValue()
    metadata['scaleFactor'] = raster.GetScale()
    
    array = np.zeros((array_shape[0],array_shape[1],dataset.RasterCount),'uint8') #pre-allocate stackedArray matrix
    
    if metadata['bands'] == 1:
        raster = dataset.GetRasterBand(1)
        metadata['noDataValue'] = raster.GetNoDataValue()
        metadata['scaleFactor'] = raster.GetScale()
              
        array = dataset.GetRasterBand(1).ReadAsArray(0,0,metadata['array_cols'],metadata['array_rows']).astype(np.float)
        array[np.where(array==metadata['noDataValue'])]=np.nan
        array = array/metadata['scaleFactor']
    
    elif metadata['bands'] > 1:    
        for i in range(1, dataset.RasterCount+1):
            band = float(dataset.GetRasterBand(i).ReadAsArray(0,0,metadata['array_cols'],metadata['array_rows']))
            band[np.where(band==metadata['noDataValue'])]=np.nan
            band = band/metadata['scaleFactor']
            array[...,i-1] = band

    return array, metadata

In [ ]:
# Read in geotiff
teak_chm = ''

In [ ]:
# extract filename for later use
just_chm_file = os.path.basename(teak_chm)
just_chm_file_split = just_chm_file.split(sep="_")

In [ ]:
chm_array, chm_array_metadata = raster2array(teak_chm)

In [ ]:
# Plot CHM
plt.figure(1)

plot_band_array(chm_array,
               chm_array_metadata['extent'],
               'Canopy Height Model',
               'Canopy Height (m)',
               'Greens',[0, 9])

plt.savefig('../output/'+ just_chm_file[0:-4]+'.png',
            dpi=300,
            orientation='landscape',
            bbox_inches='tight',
            pad_inches=0.1)

In [ ]:
# Extract spatial information of dataset
chm_mapinfo = teak_chm_dataset.GetGeoTransform()
chm_mapinfo

In [ ]:
xMin = chm_mapinfo[0]
yMax = chm_mapinfo[3]

xMax = xMin + cols/chm_mapinfo[1]
yMin = yMax + rows/chm_mapinfo[5]

chm_ext = (xMin,xMax,yMin,yMax)
print('chm raster extent:',chm_ext)

In [ ]:
# Calculate statistics for dataset
chm_stats = chm_array.GetStatistics(True,True)
print('TEAK CHM Statistics: Minimum = %.2f, Maximum = %.2f, Mean = %.3f, StDev = %.3f'
     %(chm_stats[0],chm_stats[1],chm_stats[2],chm_stats[3]))

In [ ]:
# Convert no data values to NaN
chm_array[chm_array == int(noDataVal)]=np.nan

In [ ]:
print(chm_array)

In [ ]:
# Determine shape of array
chm_array.shape

In [ ]:
# Determine the percent NaN and NonZero Data Values
pct_nan = np.count_nonzero(np.isnan(chm_array))/(rows*cols)
print('Percent NaN:',round(pct_nan*100,2))
print('Percent non-zero:',round(100*np.count_nonzero(chm_array)/(rows*cols),2))

In [ ]:
# %load ../functions/neon_aop_lidar_python_functions/plot_spatial_array.py
def plot_spatial_array(band_array,spatial_extent,colorlimit,ax=plt.gca(),title='',cmap_title='',colormap='greys'):
    plot = plt.imshow(band_array,extent=spatial_extent,clim=colorlimit); 
    cbar = plt.colorbar(plot,aspect=40); plt.set_cmap(colormap); 
    cbar.set_label(cmap_title,rotation=90,labelpad=20);
    plt.title(title); ax = plt.gca(); 
    ax.ticklabel_format(useOffset=False, style='plain'); #do not use scientific notation #
    rotatexlabels = plt.setp(ax.get_xticklabels(),rotation=90); #rotate x tick labels 90 degrees

In [ ]:
# First plot of Canopy Height Model
plot_spatial_array(chm_array,chm_ext, colorlimit = [np.min(chm_array),np.max(chm_array)],colormap='RdYlGn')

In [ ]:
# Plot a histogram of the canopy height model with 100 bins
plt.hist(chm_array[~np.isnan(chm_array)],100);
ax = plt.gca()
ax.set_ylim([0,15000])
plt.title('Distribution of TEAK Canopy Height')
plt.xlabel('Tree Height (m)')
plt.ylabel('Relative Frequency')

In [ ]:
# Plot Canopy Height Model with heights above 35 meters masked out
plot_spatial_array(chm_array,chm_ext,(0,35),title='TEAK Canopy Height',cmap_title='Canopy Height, m',colormap = 'BuGn')

### Threshold Based Raster Classification

In [ ]:
# Classify canopy height model into five 10m classes
chm_reclass = copy.copy(chm_array)
chm_reclass[np.where(chm_array==0)]=1
chm_reclass[np.where((chm_array > 0) &(chm_array <= 10))] = 2
chm_reclass[np.where((chm_array > 10) &(chm_array <= 20))] = 3
chm_reclass[np.where((chm_array > 20) &(chm_array <= 30))] = 4
chm_reclass[np.where(chm_array > 30)] = 5

In [ ]:
# Plot classified canopy height model
import matplotlib.colors as colors
plt.figure()
cmapCHM = colors.ListedColormap(['lightblue','yellow','orange','green','red'])
plt.imshow(chm_reclass,extent=chm_ext,cmap=cmapCHM)
plt.title('TEAK CHM Classification')
ax = plt.gca()
ax.ticklabel_format(useOffset=False, style='plain')
rotatexlabels = plt.setp(ax.get_xticklabels(),rotation=90)

import matplotlib.patches as mpatches
class1_box = mpatches.Patch(color='lightblue', label='CHM = 0m')
class2_box = mpatches.Patch(color='yellow', label='0m < CHM <= 10m')
class3_box = mpatches.Patch(color='orange', label='10m < CHM <= 20m')
class4_box = mpatches.Patch(color='green', label='20m < CHM <= 30m')
class5_box = mpatches.Patch(color='red', label='CHM > 30m')

ax.legend(handles = [class1_box,class2_box,class3_box,class4_box,class5_box], handlelength = 0.7, bbox_to_anchor = (1.05, 0.4),loc='lower left',borderaxespad=0.)
